In [1]:
!pip install pyspark findspark

In [2]:
import findspark
findspark.init()

In [3]:
from pyspark.sql import SparkSession
import pyspark.sql.functions as F

In [4]:
spark = (
    SparkSession.builder.master('local[*]')
    .appName('Aula de Alias')
    .config('spark.sql.legacy.timeParserPolicy', 'LEGACY')
    .getOrCreate()
)

24/05/02 11:10:14 WARN Utils: Your hostname, casa-Default-string resolves to a loopback address: 127.0.1.1; using 192.168.0.9 instead (on interface enp7s0)
24/05/02 11:10:14 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address


Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


24/05/02 11:10:15 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
24/05/02 11:10:15 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.
24/05/02 11:10:15 WARN Utils: Service 'SparkUI' could not bind on port 4041. Attempting port 4042.


In [5]:
spark

In [6]:
df = spark.read.parquet('./DATASETS/LOGINS.parquet')

In [15]:
df.groupBy(F.col('estado').alias('UF')).agg(F.count('*').alias('qntd')).show(20, False)

+---+----+
|UF |qntd|
+---+----+
|SC |44  |
|RO |39  |
|PI |35  |
|AM |34  |
|RR |33  |
|GO |45  |
|TO |34  |
|MT |29  |
|SP |49  |
|ES |38  |
|PB |34  |
|RS |39  |
|MS |42  |
|AL |36  |
|MG |46  |
|PA |28  |
|BA |42  |
|SE |44  |
|PE |34  |
|CE |45  |
+---+----+
only showing top 20 rows



In [16]:
df1 = df.alias('loggins')

In [17]:
df1.show()

+--------------+--------------------+----------+------------------+------+-------------+---------------+------------+--------------------+-------------------+
|           cpf|               email|     senha|data_de_nascimento|estado|data_cadastro|           ipv4|cor_favorita|           profissao|           telefone|
+--------------+--------------------+----------+------------------+------+-------------+---------------+------------+--------------------+-------------------+
|981.507.362-12|pedro-lucas53@gma...|+7^7E%xFBc|        2006-12-18|    RR|   2023-02-26| 99.107.250.210|        Roxo|    Jogador De Golfe|       31 7785-4046|
|493.705.168-75|rezendeisaac@hotm...|_O_2GRnGOe|        1992-06-17|    GO|   2023-02-16|  197.11.26.213|       Ciano|Atleta De Arremes...|    (031) 0803-6753|
|398.471.625-73|felipepires@uol.c...|*Aw5EOAvy9|        1921-11-11|    MG|   2023-01-02|   181.90.63.58|        Azul|      Papiloscopista|       11 9674-0553|
|092.618.354-06|stellamoraes@bol....|mw0AWYAs#

In [19]:
livros = spark.read.parquet('./DATASETS/LIVROS.parquet').alias('livros')
compras = spark.read.parquet('./DATASETS/COMPRAS.parquet').alias('compras')

In [20]:
df2 = compras.join(livros, F.col('cd_livro') == F.col('livros.id'), 'inner')

In [22]:
df2.select('livros.*').show()

+---------+------------------+---------------+-------------+-------------+--------------+------+
|       id|      cnpj_editora|data_lancamento|          ean|       isbn10|numero_paginas| preco|
+---------+------------------+---------------+-------------+-------------+--------------+------+
|030334762|80.463.719/0001-06|     2013-05-08|7122725133429|0-85193-340-8|           099|123.47|
|013721981|76.351.940/0001-97|     2004-01-27|8786391051693|1-60705-523-6|           379|133.43|
|019458805|81.253.490/0001-30|     2007-12-13|2628486550239|0-916523-25-X|           788|238.54|
|014347542|76.415.890/0001-64|     2007-04-05|3883750237349|0-8360-5203-X|           452|182.43|
|020215846|27.659.813/0001-44|     2014-03-25|7684001180416|0-216-16469-9|           794|155.18|
|010325500|27.689.540/0001-80|     2000-09-09|8793472196570|0-933627-98-X|           306| 30.62|
|053479015|54.327.896/0001-47|     2018-12-21|1750535865566|0-518-04786-5|           105|207.52|
|012331534|84.317.925/0001-50|

In [23]:
df2.show()

+------+---------------------+-------------------+--------------------+----------+-------------------------+----------+--------+---------------+--------------------+-----------+---------+----------+---------+------------------+---------------+-------------+-------------+--------------+------+
|    id|cartao_data_expiracao|      cartao_numero|     cartao_bandeira|cartao_cvc|codigo_transacao_bancaria|      data|    hora|           ipv4|                ipv6|cep_entrega| cd_livro|cd_cliente|       id|      cnpj_editora|data_lancamento|          ean|       isbn10|numero_paginas| preco|
+------+---------------------+-------------------+--------------------+----------+-------------------------+----------+--------+---------------+--------------------+-----------+---------+----------+---------+------------------+---------------+-------------+-------------+--------------+------+
|012389|                11/25|   5500804500517692|            Discover|       959|     GB98MPIH622108593...|2021-07-24

In [26]:
df2.select('compras.id').show()

+------+
|    id|
+------+
|012389|
|012476|
|012478|
|012534|
|012549|
|012574|
|012579|
|012648|
|012675|
|012698|
|012845|
|012948|
|012964|
|012984|
|012986|
|013257|
|013265|
|013274|
|013457|
|013478|
+------+
only showing top 20 rows



In [29]:
df2.select(F.col('livros.*'), F.col('compras.data')).show()

+---------+------------------+---------------+-------------+-------------+--------------+------+----------+
|       id|      cnpj_editora|data_lancamento|          ean|       isbn10|numero_paginas| preco|      data|
+---------+------------------+---------------+-------------+-------------+--------------+------+----------+
|030334762|80.463.719/0001-06|     2013-05-08|7122725133429|0-85193-340-8|           099|123.47|2021-07-24|
|013721981|76.351.940/0001-97|     2004-01-27|8786391051693|1-60705-523-6|           379|133.43|2021-08-23|
|019458805|81.253.490/0001-30|     2007-12-13|2628486550239|0-916523-25-X|           788|238.54|2020-01-27|
|014347542|76.415.890/0001-64|     2007-04-05|3883750237349|0-8360-5203-X|           452|182.43|2021-07-15|
|020215846|27.659.813/0001-44|     2014-03-25|7684001180416|0-216-16469-9|           794|155.18|2020-07-22|
|010325500|27.689.540/0001-80|     2000-09-09|8793472196570|0-933627-98-X|           306| 30.62|2020-05-19|
|053479015|54.327.896/0001-4